In [ ]:
```xml
<VSCode.Cell language="markdown">
# Service Decomposition Patterns

Strategies for breaking down monoliths into microservices using Domain-Driven Design and proven migration patterns.

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    DECOMPOSITION APPROACHES                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   BUSINESS CAPABILITY          SUBDOMAIN (DDD)          DATA-DRIVEN         │
│   ────────────────            ───────────────           ──────────          │
│   "What the business does"    "Bounded Contexts"        "Data ownership"    │
│                                                                              │
│   ┌─────────────┐             ┌─────────────┐           ┌─────────────┐     │
│   │ Sales       │             │ Order       │           │ Orders DB   │     │
│   │ Capability  │             │ Context     │           │ ───────     │     │
│   ├─────────────┤             ├─────────────┤           │ order_svc   │     │
│   │ Inventory   │             │ Inventory   │           ├─────────────┤     │
│   │ Capability  │             │ Context     │           │ Users DB    │     │
│   ├─────────────┤             ├─────────────┤           │ ───────     │     │
│   │ Shipping    │             │ Shipping    │           │ user_svc    │     │
│   │ Capability  │             │ Context     │           └─────────────┘     │
│   └─────────────┘             └─────────────┘                               │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Decomposition Strategies Comparison

| Strategy | Description | Best For | Challenges |
|----------|-------------|----------|------------|
| **By Business Capability** | Align with what business does | Stable domains | May not match tech boundaries |
| **By Subdomain (DDD)** | Bounded contexts from domain model | Complex domains | Requires DDD expertise |
| **By Data Ownership** | Each service owns its data | Data-heavy apps | Shared data challenges |
| **By Team Structure** | Conway's Law alignment | Large orgs | May create artificial boundaries |
| **By Change Frequency** | Group by release cadence | Varied volatility | May ignore domain cohesion |
| **By Scalability Needs** | Separate scaling requirements | Performance-critical | May over-fragment |
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Domain-Driven Design (DDD) Fundamentals

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    DDD STRATEGIC PATTERNS                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  DOMAIN: The subject area your software addresses (e.g., e-commerce)        │
│                                                                              │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │                         PROBLEM SPACE                                │    │
│  │                                                                      │    │
│  │   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐              │    │
│  │   │   Core      │   │ Supporting  │   │  Generic   │              │    │
│  │   │  Subdomain  │   │  Subdomain  │   │ Subdomain  │              │    │
│  │   │ ─────────── │   │ ─────────── │   │ ────────── │              │    │
│  │   │ Competitive │   │ Necessary   │   │ Commodity  │              │    │
│  │   │ advantage   │   │ but not     │   │ (buy/use   │              │    │
│  │   │ (build!)    │   │ differentiator│ │  existing) │              │    │
│  │   └─────────────┘   └─────────────┘   └─────────────┘              │    │
│  └─────────────────────────────────────────────────────────────────────┘    │
│                                      │                                       │
│                                      ▼                                       │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │                        SOLUTION SPACE                                │    │
│  │                                                                      │    │
│  │    BOUNDED CONTEXT: Explicit boundary with its own ubiquitous       │    │
│  │    language, models, and code. Maps to microservices.               │    │
│  │                                                                      │    │
│  │   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐              │    │
│  │   │  Orders     │   │  Catalog    │   │  Payments   │              │    │
│  │   │  Context    │   │  Context    │   │  Context    │              │    │
│  │   │ ─────────── │   │ ─────────── │   │ ─────────── │              │    │
│  │   │ Order       │   │ Product     │   │ Payment     │              │    │
│  │   │ OrderLine   │   │ Category    │   │ Transaction │              │    │
│  │   │ Customer    │   │ Pricing     │   │ Refund      │              │    │
│  │   └─────────────┘   └─────────────┘   └─────────────┘              │    │
│  └─────────────────────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Bounded Context Relationships

```
CONTEXT MAP: Shows how bounded contexts relate to each other

┌──────────────────┐                      ┌──────────────────┐
│   ORDER CONTEXT  │                      │  SHIPPING        │
│   ─────────────  │                      │  CONTEXT         │
│                  │    Customer/         │  ─────────       │
│  Order           │    Supplier          │                  │
│  Customer ───────┼────────────────────▶ │  Shipment        │
│  Address         │                      │  Recipient       │
│                  │                      │  Address         │
└──────────────────┘                      └──────────────────┘
        │                                          ▲
        │                                          │
        │ Shared Kernel                   Anticorruption
        │ (shared models)                 Layer (ACL)
        ▼                                          │
┌──────────────────┐                      ┌────────┴─────────┐
│  INVENTORY       │                      │  EXTERNAL        │
│  CONTEXT         │                      │  SHIPPING API    │
│  ────────        │                      │  ─────────────   │
│                  │                      │  (FedEx, UPS)    │
│  Product         │                      │                  │
│  StockLevel      │                      │                  │
└──────────────────┘                      └──────────────────┘


RELATIONSHIP PATTERNS:
┌────────────────────┬─────────────────────────────────────────┐
│ Shared Kernel      │ Two contexts share a common model       │
├────────────────────┼─────────────────────────────────────────┤
│ Customer/Supplier  │ Upstream provides, downstream consumes  │
├────────────────────┼─────────────────────────────────────────┤
│ Conformist         │ Downstream conforms to upstream model   │
├────────────────────┼─────────────────────────────────────────┤
│ Anti-corruption    │ Translation layer to protect from       │
│ Layer (ACL)        │ external/legacy models                  │
├────────────────────┼─────────────────────────────────────────┤
│ Open Host Service  │ Well-defined protocol for integration   │
├────────────────────┼─────────────────────────────────────────┤
│ Published Language │ Shared schema (e.g., events, API spec)  │
└────────────────────┴─────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## DDD Tactical Patterns (Building Blocks)

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    TACTICAL PATTERNS                                         │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  ENTITY                          VALUE OBJECT                               │
│  ──────                          ────────────                               │
│  Identity matters                Identity doesn't matter                    │
│  Mutable                         Immutable                                  │
│  Examples: User, Order           Examples: Money, Address, DateRange        │
│                                                                              │
│  AGGREGATE                       AGGREGATE ROOT                             │
│  ─────────                       ──────────────                             │
│  Cluster of entities/VOs         Entry point to aggregate                   │
│  Transactional boundary          Enforces invariants                        │
│  Consistency unit                Only root is referenced externally         │
│                                                                              │
│    ┌─────────────────────────────────────────┐                              │
│    │          ORDER AGGREGATE                │                              │
│    │  ┌───────────────────────────────────┐  │                              │
│    │  │ Order (Aggregate Root)            │  │                              │
│    │  │  - orderId: OrderId               │  │                              │
│    │  │  - status: OrderStatus            │  │                              │
│    │  │  - placedAt: DateTime             │  │                              │
│    │  └───────────────────────────────────┘  │                              │
│    │              │                          │                              │
│    │              ▼                          │                              │
│    │  ┌─────────────────┐  ┌──────────────┐  │                              │
│    │  │  OrderLine      │  │   Money      │  │                              │
│    │  │  (Entity)       │  │ (Value Obj)  │  │                              │
│    │  └─────────────────┘  └──────────────┘  │                              │
│    └─────────────────────────────────────────┘                              │
│                                                                              │
│  REPOSITORY                      DOMAIN SERVICE                             │
│  ──────────                      ──────────────                             │
│  Persistence abstraction         Business logic that doesn't                │
│  Returns aggregates              belong to an entity                        │
│                                                                              │
│  DOMAIN EVENT                    FACTORY                                    │
│  ────────────                    ───────                                    │
│  Something that happened         Complex object creation                    │
│  in the domain                   Encapsulates construction                  │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="python">
# DDD Tactical Patterns: Python Implementation

from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Optional
from uuid import UUID, uuid4
from enum import Enum
from abc import ABC, abstractmethod

# ============ VALUE OBJECTS ============
@dataclass(frozen=True)  # Immutable
class Money:
    """Value Object: Identity doesn't matter, immutability enforced."""
    amount: float
    currency: str = "USD"
    
    def add(self, other: "Money") -> "Money":
        if self.currency != other.currency:
            raise ValueError("Currency mismatch")
        return Money(self.amount + other.amount, self.currency)
    
    def multiply(self, factor: float) -> "Money":
        return Money(round(self.amount * factor, 2), self.currency)

@dataclass(frozen=True)
class Address:
    """Value Object: Immutable address."""
    street: str
    city: str
    postal_code: str
    country: str

# ============ ENTITIES ============
@dataclass
class OrderLine:
    """Entity within Order aggregate."""
    id: UUID
    product_id: UUID
    quantity: int
    unit_price: Money
    
    @property
    def total(self) -> Money:
        return self.unit_price.multiply(self.quantity)

# ============ DOMAIN EVENTS ============
class DomainEvent(ABC):
    """Base class for domain events."""
    occurred_at: datetime = field(default_factory=datetime.utcnow)

@dataclass
class OrderPlaced(DomainEvent):
    order_id: UUID
    customer_id: UUID
    total: Money
    occurred_at: datetime = field(default_factory=datetime.utcnow)

@dataclass
class OrderShipped(DomainEvent):
    order_id: UUID
    tracking_number: str
    occurred_at: datetime = field(default_factory=datetime.utcnow)

# ============ AGGREGATE ROOT ============
class OrderStatus(Enum):
    DRAFT = "draft"
    PLACED = "placed"
    PAID = "paid"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"

@dataclass
class Order:
    """
    Aggregate Root: Entry point for Order aggregate.
    Enforces invariants and controls access to child entities.
    """
    id: UUID
    customer_id: UUID
    status: OrderStatus = OrderStatus.DRAFT
    lines: List[OrderLine] = field(default_factory=list)
    shipping_address: Optional[Address] = None
    _events: List[DomainEvent] = field(default_factory=list)
    
    # Business invariant: order must have items to be placed
    def place(self) -> None:
        if not self.lines:
            raise ValueError("Cannot place empty order")
        if self.status != OrderStatus.DRAFT:
            raise ValueError(f"Cannot place order in {self.status} status")
        if not self.shipping_address:
            raise ValueError("Shipping address required")
        
        self.status = OrderStatus.PLACED
        self._events.append(OrderPlaced(
            order_id=self.id,
            customer_id=self.customer_id,
            total=self.total
        ))
    
    def add_line(self, product_id: UUID, quantity: int, unit_price: Money) -> None:
        if self.status != OrderStatus.DRAFT:
            raise ValueError("Cannot modify placed order")
        
        line = OrderLine(
            id=uuid4(),
            product_id=product_id,
            quantity=quantity,
            unit_price=unit_price
        )
        self.lines.append(line)
    
    @property
    def total(self) -> Money:
        if not self.lines:
            return Money(0)
        return Money(
            sum(line.total.amount for line in self.lines),
            self.lines[0].unit_price.currency
        )
    
    def collect_events(self) -> List[DomainEvent]:
        events = self._events.copy()
        self._events.clear()
        return events

# ============ REPOSITORY INTERFACE ============
class OrderRepository(ABC):
    """Repository: Abstracts persistence."""
    
    @abstractmethod
    def save(self, order: Order) -> None:
        pass
    
    @abstractmethod
    def find_by_id(self, order_id: UUID) -> Optional[Order]:
        pass

# ============ DOMAIN SERVICE ============
class PricingService:
    """Domain Service: Logic that doesn't belong to an entity."""
    
    def calculate_discount(self, order: Order, customer_tier: str) -> Money:
        discount_rates = {"bronze": 0.0, "silver": 0.05, "gold": 0.10}
        rate = discount_rates.get(customer_tier, 0.0)
        return order.total.multiply(rate)

# ============ USAGE EXAMPLE ============
# Create order aggregate
order = Order(id=uuid4(), customer_id=uuid4())
order.shipping_address = Address("123 Main St", "NYC", "10001", "USA")
order.add_line(uuid4(), quantity=2, unit_price=Money(29.99))
order.add_line(uuid4(), quantity=1, unit_price=Money(49.99))

print(f"Order total: ${order.total.amount:.2f}")
print(f"Order status: {order.status.value}")

# Place the order
order.place()
print(f"After placing: {order.status.value}")

# Collect domain events for publishing
events = order.collect_events()
for event in events:
    print(f"Event: {type(event).__name__}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Aggregate Design Rules

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    AGGREGATE DESIGN RULES                                    │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  RULE 1: PROTECT BUSINESS INVARIANTS                                        │
│  ─────────────────────────────────────                                       │
│  All changes go through aggregate root                                       │
│                                                                              │
│      ✗ BAD                        ✓ GOOD                                    │
│      order.lines[0].qty = 5       order.update_line_quantity(line_id, 5)    │
│                                                                              │
│  RULE 2: DESIGN SMALL AGGREGATES                                            │
│  ────────────────────────────────                                            │
│  Smaller = better concurrency, less lock contention                         │
│                                                                              │
│      ✗ BAD                        ✓ GOOD                                    │
│      Customer                     Customer                                  │
│        └─ Orders (1000s)            └─ CustomerProfile                      │
│             └─ OrderLines         Order (separate aggregate)                │
│                                     └─ OrderLines                           │
│                                                                              │
│  RULE 3: REFERENCE BY IDENTITY, NOT OBJECT                                  │
│  ─────────────────────────────────────────                                   │
│  Cross-aggregate references use IDs, not object references                  │
│                                                                              │
│      ✗ BAD                        ✓ GOOD                                    │
│      order.customer = Customer()  order.customer_id = UUID                  │
│                                                                              │
│  RULE 4: USE EVENTUAL CONSISTENCY ACROSS AGGREGATES                         │
│  ────────────────────────────────────────────────────                        │
│  Only one aggregate modified per transaction                                 │
│  Use domain events for cross-aggregate updates                              │
│                                                                              │
│      Transaction 1: Order.place() → publishes OrderPlaced                   │
│      Transaction 2: Inventory handles OrderPlaced → reserves stock          │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Identifying Bounded Contexts: Event Storming

```
EVENT STORMING: Collaborative workshop to discover domain events and boundaries

STEP 1: Collect Domain Events (orange stickies)
─────────────────────────────────────────────────────────────────────────────
│ Order      │ Payment   │ Payment  │ Order    │ Shipment  │ Order      │
│ Placed     │ Initiated │ Completed│ Confirmed│ Created   │ Delivered  │
└────────────┴───────────┴──────────┴──────────┴───────────┴────────────┘

STEP 2: Add Commands (blue) and Actors (yellow)
─────────────────────────────────────────────────────────────────────────────
  [Customer]         [Payment System]        [Warehouse Staff]
      │                    │                       │
      ▼                    ▼                       ▼
  ┌─────────┐         ┌──────────┐           ┌──────────┐
  │ Place   │         │ Process  │           │ Ship     │
  │ Order   │         │ Payment  │           │ Order    │
  └────┬────┘         └─────┬────┘           └─────┬────┘
       │                    │                      │
       ▼                    ▼                      ▼
  [Order Placed]    [Payment Completed]    [Shipment Created]


STEP 3: Group into Bounded Contexts (draw boundaries)
─────────────────────────────────────────────────────────────────────────────

┌─────────────────────────────┐  ┌─────────────────────────────┐
│     ORDER CONTEXT           │  │     PAYMENT CONTEXT         │
│  ─────────────────          │  │  ────────────────           │
│  Events:                    │  │  Events:                    │
│    - Order Placed           │  │    - Payment Initiated      │
│    - Order Confirmed        │  │    - Payment Completed      │
│    - Order Cancelled        │  │    - Payment Failed         │
│                             │  │    - Refund Issued          │
│  Aggregates:                │  │                             │
│    - Order                  │  │  Aggregates:                │
│                             │  │    - Payment                │
└─────────────────────────────┘  └─────────────────────────────┘

┌─────────────────────────────┐  ┌─────────────────────────────┐
│    SHIPPING CONTEXT         │  │    INVENTORY CONTEXT        │
│  ──────────────────         │  │  ──────────────────         │
│  Events:                    │  │  Events:                    │
│    - Shipment Created       │  │    - Stock Reserved         │
│    - Shipment In Transit    │  │    - Stock Released         │
│    - Order Delivered        │  │    - Low Stock Alert        │
│                             │  │                             │
│  Aggregates:                │  │  Aggregates:                │
│    - Shipment               │  │    - InventoryItem          │
└─────────────────────────────┘  └─────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="python">
# Event Storming to Service Mapping

from dataclasses import dataclass, field
from typing import List, Set, Dict

@dataclass
class DomainEventDefinition:
    name: str
    triggered_by: str  # Command or event
    triggers: List[str] = field(default_factory=list)  # Downstream effects

@dataclass
class BoundedContext:
    name: str
    events: List[DomainEventDefinition]
    aggregates: List[str]
    commands: List[str]
    
    @property
    def inbound_events(self) -> Set[str]:
        """Events this context reacts to from other contexts."""
        internal = {e.name for e in self.events}
        triggers = {t for e in self.events for t in e.triggers}
        return triggers - internal
    
    @property
    def outbound_events(self) -> Set[str]:
        """Events this context publishes."""
        return {e.name for e in self.events}

def map_to_microservices(contexts: List[BoundedContext]) -> Dict:
    """Map bounded contexts to microservices."""
    services = {}
    for ctx in contexts:
        services[f"{ctx.name.lower().replace(' ', '_')}_service"] = {
            "bounded_context": ctx.name,
            "aggregates": ctx.aggregates,
            "api_endpoints": [f"/{cmd.lower().replace(' ', '-')}" for cmd in ctx.commands],
            "publishes": list(ctx.outbound_events),
            "subscribes_to": list(ctx.inbound_events),
        }
    return services

# Define bounded contexts from event storming
contexts = [
    BoundedContext(
        name="Order",
        events=[
            DomainEventDefinition("OrderPlaced", "PlaceOrder", ["ReserveStock", "InitiatePayment"]),
            DomainEventDefinition("OrderConfirmed", "PaymentCompleted", ["CreateShipment"]),
            DomainEventDefinition("OrderCancelled", "CancelOrder", ["ReleaseStock", "RefundPayment"]),
        ],
        aggregates=["Order", "OrderLine"],
        commands=["Place Order", "Cancel Order", "Get Order"]
    ),
    BoundedContext(
        name="Payment",
        events=[
            DomainEventDefinition("PaymentInitiated", "OrderPlaced", []),
            DomainEventDefinition("PaymentCompleted", "ProcessPayment", ["OrderConfirmed"]),
            DomainEventDefinition("PaymentFailed", "ProcessPayment", ["OrderCancelled"]),
            DomainEventDefinition("RefundIssued", "OrderCancelled", []),
        ],
        aggregates=["Payment", "Refund"],
        commands=["Process Payment", "Issue Refund"]
    ),
    BoundedContext(
        name="Inventory",
        events=[
            DomainEventDefinition("StockReserved", "OrderPlaced", []),
            DomainEventDefinition("StockReleased", "OrderCancelled", []),
            DomainEventDefinition("LowStockAlert", "StockReserved", []),
        ],
        aggregates=["InventoryItem", "Reservation"],
        commands=["Reserve Stock", "Release Stock", "Check Availability"]
    ),
    BoundedContext(
        name="Shipping",
        events=[
            DomainEventDefinition("ShipmentCreated", "OrderConfirmed", []),
            DomainEventDefinition("ShipmentInTransit", "MarkShipped", []),
            DomainEventDefinition("OrderDelivered", "ConfirmDelivery", []),
        ],
        aggregates=["Shipment", "TrackingInfo"],
        commands=["Create Shipment", "Update Tracking", "Confirm Delivery"]
    ),
]

# Map to microservices
services = map_to_microservices(contexts)

print("=== Microservices from Bounded Contexts ===\n")
for name, config in services.items():
    print(f"📦 {name}")
    print(f"   Aggregates: {', '.join(config['aggregates'])}")
    print(f"   APIs: {config['api_endpoints']}")
    print(f"   Publishes: {config['publishes']}")
    print(f"   Subscribes: {config['subscribes_to']}")
    print()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## The Strangler Fig Pattern

Incremental migration strategy to replace a monolith piece by piece.

```
STRANGLER FIG MIGRATION PHASES:

PHASE 1: Facade                   PHASE 2: Parallel Run
─────────────────                 ────────────────────

    ┌─────────────┐                   ┌─────────────┐
    │   Facade    │                   │   Facade    │
    │  (Router)   │                   │  (Router)   │
    └──────┬──────┘                   └──────┬──────┘
           │                                 │
           ▼                          ┌──────┴──────┐
    ┌─────────────┐                   ▼             ▼
    │  MONOLITH   │              ┌─────────┐  ┌─────────┐
    │             │              │ New Svc │  │MONOLITH │
    │  Feature A  │              │Feature A│  │         │
    │  Feature B  │              └─────────┘  │Feature B│
    │  Feature C  │                           │Feature C│
    └─────────────┘                           └─────────┘


PHASE 3: Traffic Shift            PHASE 4: Completion
──────────────────                ────────────────────

    ┌─────────────┐                   ┌─────────────┐
    │   Facade    │                   │ API Gateway │
    └──────┬──────┘                   └──────┬──────┘
           │                                 │
    ┌──────┴──────┐               ┌──────────┼──────────┐
    ▼             ▼               ▼          ▼          ▼
┌─────────┐  ┌─────────┐     ┌─────────┐┌─────────┐┌─────────┐
│ New Svc │  │MONOLITH │     │ Svc A   ││ Svc B   ││ Svc C   │
│Feature A│  │(smaller)│     └─────────┘└─────────┘└─────────┘
│Feature B│  │Feature C│
└─────────┘  └─────────┘          Monolith retired!
```
</VSCode.Cell>
<VSCode.Cell language="python">
# Strangler Fig Pattern Implementation

from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum
from typing import Dict, Any, Callable
import random

class TrafficMode(Enum):
    MONOLITH = "monolith"           # 100% to monolith
    SHADOW = "shadow"               # Copy to new, use monolith response
    CANARY = "canary"               # Small % to new service
    SPLIT = "split"                 # Configurable split
    NEW_SERVICE = "new_service"     # 100% to new service

@dataclass
class RoutingConfig:
    feature: str
    mode: TrafficMode
    new_service_percentage: int = 0  # 0-100

class StranglerFacade:
    """
    Routes traffic between monolith and new microservices.
    Supports gradual migration with traffic splitting.
    """
    
    def __init__(self):
        self.routing_config: Dict[str, RoutingConfig] = {}
        self.monolith: Callable = None
        self.new_services: Dict[str, Callable] = {}
        self.metrics: Dict[str, Dict] = {}
    
    def register_monolith(self, handler: Callable):
        self.monolith = handler
    
    def register_new_service(self, feature: str, handler: Callable):
        self.new_services[feature] = handler
    
    def configure_routing(self, config: RoutingConfig):
        self.routing_config[config.feature] = config
        self.metrics[config.feature] = {
            "monolith_calls": 0,
            "new_service_calls": 0,
            "shadow_mismatches": 0
        }
    
    def route(self, feature: str, request: Dict[str, Any]) -> Dict[str, Any]:
        config = self.routing_config.get(feature)
        
        if not config or config.mode == TrafficMode.MONOLITH:
            return self._call_monolith(feature, request)
        
        if config.mode == TrafficMode.NEW_SERVICE:
            return self._call_new_service(feature, request)
        
        if config.mode == TrafficMode.SHADOW:
            return self._shadow_call(feature, request)
        
        if config.mode in (TrafficMode.CANARY, TrafficMode.SPLIT):
            return self._split_traffic(feature, request, config.new_service_percentage)
        
        return self._call_monolith(feature, request)
    
    def _call_monolith(self, feature: str, request: Dict) -> Dict:
        self.metrics[feature]["monolith_calls"] += 1
        return self.monolith(feature, request)
    
    def _call_new_service(self, feature: str, request: Dict) -> Dict:
        self.metrics[feature]["new_service_calls"] += 1
        return self.new_services[feature](request)
    
    def _shadow_call(self, feature: str, request: Dict) -> Dict:
        """Call both, compare results, return monolith response."""
        monolith_response = self._call_monolith(feature, request)
        
        if feature in self.new_services:
            new_response = self.new_services[feature](request)
            if monolith_response != new_response:
                self.metrics[feature]["shadow_mismatches"] += 1
                print(f"[SHADOW MISMATCH] {feature}: {monolith_response} vs {new_response}")
        
        return monolith_response
    
    def _split_traffic(self, feature: str, request: Dict, percentage: int) -> Dict:
        if random.randint(1, 100) <= percentage:
            return self._call_new_service(feature, request)
        return self._call_monolith(feature, request)

# === EXAMPLE USAGE ===

# Simulate monolith
def monolith_handler(feature: str, request: Dict) -> Dict:
    return {"source": "monolith", "feature": feature, "result": request.get("id", 0) * 10}

# New microservices
def order_service(request: Dict) -> Dict:
    return {"source": "order_service", "result": request.get("id", 0) * 10}

def inventory_service(request: Dict) -> Dict:
    return {"source": "inventory_service", "result": request.get("id", 0) * 10}

# Set up strangler facade
facade = StranglerFacade()
facade.register_monolith(monolith_handler)
facade.register_new_service("orders", order_service)
facade.register_new_service("inventory", inventory_service)

# Configure routing for each feature
facade.configure_routing(RoutingConfig("orders", TrafficMode.SPLIT, 50))  # 50% to new
facade.configure_routing(RoutingConfig("inventory", TrafficMode.SHADOW))   # Shadow mode
facade.configure_routing(RoutingConfig("payments", TrafficMode.MONOLITH))  # Still on monolith

# Simulate traffic
print("=== Strangler Fig Traffic Routing ===\n")
for _ in range(10):
    response = facade.route("orders", {"id": 42})
    print(f"Orders -> {response['source']}")

print(f"\n=== Metrics ===")
for feature, stats in facade.metrics.items():
    print(f"{feature}: {stats}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Anti-Corruption Layer (ACL)

```
PROBLEM: New service needs data from legacy system with incompatible model

┌───────────────────────────────────────────────────────────────────────────┐
│                                                                            │
│   LEGACY MONOLITH                        NEW ORDER SERVICE                 │
│   ───────────────                        ─────────────────                 │
│                                                                            │
│   LegacyCustomer                         Customer                          │
│   ┌─────────────────┐                    ┌─────────────────┐              │
│   │ CUST_ID: INT    │                    │ id: UUID        │              │
│   │ CUST_NM: VARCHAR│       ACL          │ name: String    │              │
│   │ ADDR_1: VARCHAR │ ◀─────────────────▶│ address: Address│              │
│   │ ADDR_2: VARCHAR │    (Translator)    │ tier: Tier      │              │
│   │ CUST_TYP: CHAR  │                    └─────────────────┘              │
│   └─────────────────┘                                                      │
│                                                                            │
└───────────────────────────────────────────────────────────────────────────┘

ACL RESPONSIBILITIES:
┌────────────────────┬─────────────────────────────────────────────────────┐
│ Data Translation   │ Convert between models (types, formats, structures)│
├────────────────────┼─────────────────────────────────────────────────────┤
│ Protocol Adaption  │ REST ↔ SOAP, sync ↔ async                          │
├────────────────────┼─────────────────────────────────────────────────────┤
│ Error Mapping      │ Translate legacy errors to domain exceptions       │
├────────────────────┼─────────────────────────────────────────────────────┤
│ Caching            │ Reduce calls to slow legacy system                 │
├────────────────────┼─────────────────────────────────────────────────────┤
│ Circuit Breaking   │ Protect from legacy system failures                │
└────────────────────┴─────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="python">
# Anti-Corruption Layer Implementation

from dataclasses import dataclass
from typing import Optional
from uuid import UUID, uuid5, NAMESPACE_DNS
from abc import ABC, abstractmethod
from enum import Enum

# === NEW DOMAIN MODEL (Clean) ===
class CustomerTier(Enum):
    BRONZE = "bronze"
    SILVER = "silver"
    GOLD = "gold"

@dataclass(frozen=True)
class Address:
    street: str
    city: str
    postal_code: str
    country: str

@dataclass
class Customer:
    id: UUID
    name: str
    email: str
    address: Address
    tier: CustomerTier

# === LEGACY MODEL (Messy) ===
@dataclass
class LegacyCustomerRecord:
    """Legacy database record with poor naming and structure."""
    CUST_ID: int
    CUST_NM: str
    EMAIL_ADDR: str
    ADDR_1: str
    ADDR_2: str
    CITY_NM: str
    ZIP_CD: str
    CNTRY_CD: str
    CUST_TYP: str  # 'B', 'S', 'G' for bronze/silver/gold

# === ANTI-CORRUPTION LAYER ===
class CustomerACL:
    """
    Anti-Corruption Layer: Translates between legacy and domain models.
    Protects new services from legacy complexity.
    """
    
    # Mapping from legacy type codes to domain enum
    TIER_MAPPING = {
        'B': CustomerTier.BRONZE,
        'S': CustomerTier.SILVER,
        'G': CustomerTier.GOLD,
    }
    
    COUNTRY_MAPPING = {
        'US': 'United States',
        'UK': 'United Kingdom',
        'CA': 'Canada',
    }
    
    @classmethod
    def translate_to_domain(cls, legacy: LegacyCustomerRecord) -> Customer:
        """Convert legacy record to clean domain model."""
        
        # Generate consistent UUID from legacy int ID
        domain_id = uuid5(NAMESPACE_DNS, f"customer-{legacy.CUST_ID}")
        
        # Combine address fields
        street = f"{legacy.ADDR_1}, {legacy.ADDR_2}".strip(", ")
        
        # Map country code
        country = cls.COUNTRY_MAPPING.get(legacy.CNTRY_CD, legacy.CNTRY_CD)
        
        # Map tier
        tier = cls.TIER_MAPPING.get(legacy.CUST_TYP, CustomerTier.BRONZE)
        
        return Customer(
            id=domain_id,
            name=legacy.CUST_NM.strip().title(),  # Clean up name
            email=legacy.EMAIL_ADDR.lower(),
            address=Address(
                street=street,
                city=legacy.CITY_NM.title(),
                postal_code=legacy.ZIP_CD,
                country=country
            ),
            tier=tier
        )
    
    @classmethod
    def translate_to_legacy(cls, customer: Customer, legacy_id: int) -> LegacyCustomerRecord:
        """Convert domain model back to legacy format for writes."""
        
        # Reverse tier mapping
        tier_code = next(
            (k for k, v in cls.TIER_MAPPING.items() if v == customer.tier),
            'B'
        )
        
        # Reverse country mapping
        country_code = next(
            (k for k, v in cls.COUNTRY_MAPPING.items() if v == customer.address.country),
            customer.address.country[:2].upper()
        )
        
        # Split address (simple heuristic)
        addr_parts = customer.address.street.split(", ", 1)
        
        return LegacyCustomerRecord(
            CUST_ID=legacy_id,
            CUST_NM=customer.name.upper(),
            EMAIL_ADDR=customer.email.upper(),
            ADDR_1=addr_parts[0] if addr_parts else "",
            ADDR_2=addr_parts[1] if len(addr_parts) > 1 else "",
            CITY_NM=customer.address.city.upper(),
            ZIP_CD=customer.address.postal_code,
            CNTRY_CD=country_code,
            CUST_TYP=tier_code
        )

# === EXAMPLE USAGE ===

# Simulate legacy data from old system
legacy_record = LegacyCustomerRecord(
    CUST_ID=12345,
    CUST_NM="JOHN DOE",
    EMAIL_ADDR="JOHN.DOE@EMAIL.COM",
    ADDR_1="123 MAIN ST",
    ADDR_2="APT 4B",
    CITY_NM="NEW YORK",
    ZIP_CD="10001",
    CNTRY_CD="US",
    CUST_TYP="G"
)

# Translate through ACL
customer = CustomerACL.translate_to_domain(legacy_record)

print("=== Anti-Corruption Layer Translation ===\n")
print("Legacy Record:")
print(f"  {legacy_record}\n")
print("Domain Model:")
print(f"  ID: {customer.id}")
print(f"  Name: {customer.name}")
print(f"  Email: {customer.email}")
print(f"  Address: {customer.address}")
print(f"  Tier: {customer.tier.value}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Decomposition Decision Framework

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    DECOMPOSITION CHECKLIST                                   │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  STEP 1: IDENTIFY BOUNDARIES                                                │
│  ──────────────────────────                                                  │
│  ☐ Run Event Storming workshops                                             │
│  ☐ Map business capabilities                                                │
│  ☐ Identify core vs supporting vs generic subdomains                        │
│  ☐ Define ubiquitous language per context                                   │
│                                                                              │
│  STEP 2: EVALUATE COUPLING                                                  │
│  ─────────────────────────                                                   │
│  ☐ Identify shared data (anti-pattern to resolve)                           │
│  ☐ Map synchronous dependencies                                             │
│  ☐ Calculate change coupling (modules that change together)                 │
│  ☐ Identify runtime coupling (services called together)                     │
│                                                                              │
│  STEP 3: DESIGN AGGREGATES                                                  │
│  ─────────────────────────                                                   │
│  ☐ Keep aggregates small                                                    │
│  ☐ Reference by ID across aggregates                                        │
│  ☐ One aggregate per transaction                                            │
│  ☐ Use eventual consistency across aggregates                               │
│                                                                              │
│  STEP 4: PLAN MIGRATION                                                     │
│  ────────────────────────                                                    │
│  ☐ Start with least coupled module                                          │
│  ☐ Use Strangler Fig pattern                                                │
│  ☐ Implement ACL for legacy integration                                     │
│  ☐ Plan data migration strategy                                             │
│                                                                              │
└─────────────────────────────────────────────────────────────────────────────┘
```
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Summary: Key Decomposition Principles

| Principle | Description |
|-----------|-------------|
| **Start with the domain** | Use DDD to find natural boundaries |
| **Single responsibility** | Each service does one thing well |
| **Loose coupling** | Services communicate via well-defined interfaces |
| **High cohesion** | Related functionality stays together |
| **Data ownership** | Each service owns its data completely |
| **Autonomous teams** | Teams own services end-to-end |
| **Design for failure** | Expect network issues, use resilience patterns |
| **Evolutionary design** | Services can be replaced or rewritten |
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Further Reading

- **Books:**
  - "Domain-Driven Design" by Eric Evans (The Blue Book)
  - "Implementing Domain-Driven Design" by Vaughn Vernon
  - "Learning Domain-Driven Design" by Vlad Khononov

- **Related Topics:**
  - [Microservices Overview](./00_overview.ipynb)
  - [Service Communication](./02_service_communication.ipynb)
  - [Distributed Patterns](./04_distributed_patterns.ipynb)
</VSCode.Cell>
```